# Hyperparameters

In [1]:
SPLIT = 0.8 # SPLIT% of train, 1 - SPLIT validation
BATCH_SIZE = 30 #seq_lenth
DROPOUT = 0
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 0.0
INPUT_SIZE = 94
HIDDEN_SIZE = 128
OUTPUT_SIZE = 94
NUM_LAYERS = 1
DROPOUT = 0
#N_LSTM_CELLS = 50

# Data Preprocessing

In [2]:
import numpy as np
import numpy as np
from numpy.random import shuffle
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import time
import copy
import pickle
import os
from torch.utils.data import Dataset, DataLoader
import PIL
from tqdm import tqdm

class one_hot_encoder:
    def __init__(self, data):
        self.classes = set(data)
        self.n_classes = len(self.classes)
        self.char_to_idx = {ch:i for i, ch in enumerate(self.classes)}
        self.idx_to_char = {i:ch for i,ch in zip(xrange(self.n_classes), self.classes)}
    
    def encode(self, string):
        """ input   string: string to be encoded to one hot encoding
            output  encoded: one hot enccoding representation of string
        """
        encoded = np.zeros((len(string), self.n_classes))
        for i,s in enumerate(string):
            idx = self.char_to_idx[s]
            encoded[i, idx] = 1
        return encoded
    
    def decode(self, indices):
        temp = [self.idx_to_char[idx] for idx in indices]
        return ''.join(temp)
    
    def check(self):
        for i in xrange(self.n_classes):
            ch = self.idx_to_char[i]
            idx = OHE.char_to_idx[ch]
            if i != idx:
                print 'OHE function is wrong, idx: %i suppose to be' %i + ch +'but' + ch + 'has idx: %i' %idx
            elif i == OHE.n_classes-1:
                print 'OHE function correct'

class MusicData(Dataset):
    def __init__(self, data_set, OHE, transform = None):
#         self.images = torch.from_numpy(data_set[:,0])
#         self.images = self.images.permute(0, 3, 1, 2)
#         self.labels = torch.Tensor(data_set[:,1])
        self.data = data_set[:-1]
        self.label = data_set[1:]
        self.transform = transform
        self.OHE = OHE
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x,y = self.data[index], self.label[index]
        x = self.OHE.encode(x)
        y = self.OHE.char_to_idx[y]
        
        y = torch.LongTensor([int(y)])
        x = torch.Tensor(x)
        sample = (x,y)
        return sample

transform = transforms.Compose([transforms.ToTensor()])
use_gpu = torch.cuda.is_available()
# class batch_generator:
#     def __init__(self, batch_size = BATCH_SIZE, n_lstm_cells = N_LSTM_CELLS):
#         self.batch_size = batch_size
#         self.seq_length = n_lstm_cells
#     def gen(self, data, num_samples, OHE):
        
#         batch_size = self.batch_size
#         seq_length = self.seq_length
#         while True:
#             curr_size = batch_size
#             X_batch = np.zeros((curr_size, seq_length, OHE.n_classes))
#             y_batch = np.zeros((curr_size, seq_length, OHE.n_classes))
#             for i in range(curr_size):
#                 idx = np.random.randint(len(data) - seq_length - 1)
#                 input_str = data[idx:idx + seq_length]
#                 target_str = data[idx + 1:idx + seq_length + 1]
#                 X_batch[i, :, :] = OHE.encode(input_str)
#                 y_batch[i, :, :] = OHE.encode(target_str)
#             yield X_batch, y_batch
# class temp_softmax:
#     def __init__(self, temp = 1.0):
#         self.temp = temp
    
#     def sample(self, soft_output):
#         ''' soft_output: the softmax output of the RNN
#         '''

In [3]:
# Read in .txt file
data = data = open('data/input.txt', 'r').read()
total_data_size = len(data)
OHE = one_hot_encoder(data)

# split into train and validation sets
train_data, validation_data = data[:int(SPLIT*total_data_size)], data[int(SPLIT*total_data_size):]
train_data_tensor, valid_data_tensor = MusicData(train_data, OHE, transform), MusicData(validation_data, OHE, transform)
# train_size, validation_size = len(train_data) - N_LSTM_CELLS + 1, len(validation_data) - N_LSTM_CELLS + 1
# batch_gen = batch_generator()

train_data_loader = DataLoader(
    dataset = train_data_tensor,
    shuffle = False,
    batch_size = BATCH_SIZE,
    num_workers = 4
)

valid_data_loader = DataLoader(
    dataset = valid_data_tensor,
    shuffle = False,
    batch_size = BATCH_SIZE,
    num_workers = 4)

# Model building

In [4]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, DROPOUT):
        super(Model, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, dropout = DROPOUT)
        
        self.linear = nn.Linear(hidden_size, output_size)
        self.hidden = self.hidden_init()
    
    def hidden_init(self):
        # check if this needs to be loaded into cuda
        if use_gpu:
            return (Variable(torch.zeros(self.num_layers, 1, self.hidden_size).cuda()),
                    Variable(torch.zeros(self.num_layers, 1, self.hidden_size).cuda()))
    
    def forward(self, inputs, hidden):
        x, new_hidden = self.lstm(inputs, hidden)
        x = self.linear(x)
        return x, new_hidden

def calc_loss_acc(model, data_loader, objective, mode):
    num_correct = Variable(torch.cuda.FloatTensor(1).zero_())
    loss_temp = 0
    total_data = 0
    model.eval()
    hidden = model.hidden_init()
    if mode == 0:
        func = 'Calculating Train Loss/Acc'
    elif mode == 1:
        func = 'Calculating Validation Loss/Acc'

    for x, y in tqdm(data_loader, desc = func):
        if use_gpu:
            x, y = Variable(x.cuda(), volatile = True), Variable(y.cuda(), volatile = True)
        output, hidden = model.forward(x, hidden)
        _,pred_y = torch.max(output.data, 2)
        #y = y.view(-1)
        #num_correct += torch.sum(torch.eq(torch.transpose(y,0,1).type(torch.cuda.LongTensor), Variable(pred_y.type(torch.cuda.LongTensor))))
        num_correct += torch.sum(torch.eq(y.type(torch.cuda.LongTensor), Variable(pred_y.type(torch.cuda.LongTensor)))).type(torch.cuda.FloatTensor)
        loss_temp += objective(output.view(torch.Size((output.size()[0],94))), y.view(-1))*x.size()[0]
        total_data += x.size()[0]
    return loss_temp/float(total_data), num_correct/float(total_data)

# Train

In [5]:
def train(model, optimizer, objective, lr_decay, epochs = 100, early_stop = 4, verbose = True):
    start_time = time.time()
    acc_record = {}
    loss_record = {}
    best_model = copy.deepcopy(model.state_dict())
    loss_temp = 0
    total_data = 0
    best_acc = last_acc = 0
    hidden = model.hidden_init()
    if use_gpu:
        model = model.cuda()
    for epoch in xrange(epochs):
        if verbose:
            print 'epoch %i' %(epoch + 1)
        model.train()
        #learning_decay.step()
        for x,y in tqdm(train_data_loader, desc = 'Training'):
            if use_gpu:
                x,y = Variable(x.cuda()), Variable(y.cuda())
            y = y.view(-1)
            # set optimizer parameter gradients to zero
            optimizer.zero_grad()

            #forward prop
            outputs, hidden = model.forward(x, hidden)
            hidden = (hidden[0].detach(), hidden[1].detach())
            _,pred_y = torch.max(outputs.data,1)
            loss = objective(outputs.view(torch.Size((outputs.size()[0],94))), y)

            #backward prop
            loss.backward()
            optimizer.step()

#             #record batch loss and num_correct
#             num_correct += torch.sum(torch.eq(y.type(torch.cuda.LongTensor), Variable(pred_y.type(torch.cuda.LongTensor)))).type(torch.cuda.FloatTensor)
#             loss_temp += loss*x.size()[0]
#             total_data += x.size()[0]
#         train_loss, train_acc = loss_temp/total_data, num_correct/total_data
        train_loss, train_acc = calc_loss_acc(model, train_data_loader, objective,0)
        valid_loss, valid_acc = calc_loss_acc(model, valid_data_loader, objective,1)
        acc_record[epoch] = [train_acc, valid_acc]
        loss_record[epoch] = [train_loss, valid_loss]
        lr_decay.step(valid_acc.data[0])
        if verbose:
            print 'train loss: %f \t valid loss: %f' %(train_loss.data[0], valid_loss.data[0])
            print 'train acc: %f \t valid acc: %f' %(train_acc.data[0], valid_acc.data[0])

        if epoch != 0 and last_acc >= valid_acc.data[0]:
            rise_epoch +=1
            if rise_epoch == early_stop:
                if verbose:
                    print 'early stop condition met'
                    time_elapsed = (time.time() - start_time)
                    print 'time elapsed: %i m %f s' %(time_elapsed//60, time_elapsed%60)
                return model, best_model, best_acc, loss_record, acc_record, (time.time() - start_time)
            last_acc = valid_acc.data[0]
        else:
            rise_epoch = 0
            if valid_acc.data[0] >= best_acc:
                best_acc = valid_acc.data[0]
                best_model = copy.deepcopy(model.state_dict())
            last_acc = valid_acc.data[0]
            if verbose:
                print '-'*80
    time_elapsed = (time.time() - start_time)
    print 'time elapsed: %i m %f s' %(time_elapsed//60, time_elapsed%60)
    return model, best_model, best_acc, loss_record, acc_record, (time.time() - start_time)

In [6]:
model_simple = Model(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE, NUM_LAYERS, DROPOUT)
optimizer = optim.SGD(model_simple.parameters(), lr=LR, momentum=MOMENTUM, weight_decay = WEIGHT_DECAY, nesterov = True)
objective = nn.CrossEntropyLoss()
lr_decay = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=2, 
                                                      verbose=False, threshold=0.0001, threshold_mode='rel', 
                                                      cooldown=0, min_lr=0, eps=1e-08)

In [7]:
model, best_model, best_acc, loss_record, acc_record, _ = train(model_simple, optimizer, objective, lr_decay, 
                                                                epochs = 100, early_stop = 4, verbose = True)

Training:   0%|          | 0/13872 [00:00<?, ?it/s]

epoch 1


Training:   0%|          | 0/13872 [00:00<?, ?it/s]

train loss: 2.507518 	 valid loss: 2.049627
train acc: 0.358033 	 valid acc: 0.418676
--------------------------------------------------------------------------------
epoch 2


Training:   0%|          | 0/13872 [00:00<?, ?it/s]

...

train loss: 1.083347 	 valid loss: 1.506351
train acc: 0.671353 	 valid acc: 0.566559
epoch 32


Training:   0%|          | 0/13872 [00:00<?, ?it/s]

train loss: 1.080388 	 valid loss: 1.505742
train acc: 0.672031 	 valid acc: 0.566348
epoch 33


Calculating Validation Loss/Acc: 100%|██████████| 3468/3468 [00:09<00:00, 354.82it/s]

train loss: 1.079710 	 valid loss: 1.505792
train acc: 0.672118 	 valid acc: 0.566281
early stop condition met
time elapsed: 65 m 42.610443 s


In [ ]:
def prepare_sequence2(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return tensor

In [ ]:
def generate(model, prime_str='$', predict_len=100, temperature=0.5):
    hidden = model.init_hidden(1)
    prime_input = prepare_sequence2(prime_str*1, letter2int)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = model(prime_input[p], hidden)
    inp = autograd.Variable(prime_input.cuda())
    inp = inp.view(1,-1)
    for p in range(predict_len):
        output, hidden = model(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        # return top_i
        # Add predicted character to string and use as next input
        temp = int2letter[top_i[0]]
        if temp=="$":
            break
        predicted += temp
        inp = autograd.Variable(prepare_sequence2(temp, letter2int).cuda())
        inp = inp.view(1,-1)

    return predicted

In [ ]:
pred = generate(model, prime_str='$', predict_len=1000, temperature=0.75)

In [ ]:
print pred